In [ ]:
#@title how much ram##
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('To enable a high-RAM runtime, select the Runtime > "Change runtime type"')
  print('menu, and then select High-RAM in the Runtime shape dropdown. Then, ')
  print('re-execute this cell.')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
#@title gpu info##
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

In [5]:
#@title Import all the libraries we need ####################################################

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import image

import numpy as np
import os
import sys
import random
from datetime import datetime
from pprint import pprint
from IPython.display import clear_output

# Data Reading
from skimage.io import imread, imsave
import pickle
import h5py

# TensorFlow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tqdm import tqdm

import h5py
from IPython.display import Image
!pip install ipynb

from datetime import datetime
import csv
import time

In [7]:
#@title download files from google drive into colab
from google.colab import drive
drive.mount('/content/drive') #after you run this a link will pop up for you to click

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
# @title Function Cell ###############################################


# keypoints function
def pklpath2tensor(pkl_file_path):
  # open a file, where you stored the pickled data
  file = open(pkl_file_path, 'rb')
  # dump information to that file
  data = pickle.load(file)
  # close the file
  file.close()

  keypoints3d = data['keypoints3d']
  keypoints3d_5Hz = keypoints3d[0::12,:,:]
  keypoints_tensor = tf.convert_to_tensor(keypoints3d_5Hz)#, dtype=None, dtype_hint=None, name=None)
  return keypoints_tensor

def vidpath2picklepath(folder, filename):
  string = os.path.basename(os.path.normpath(filename))
  for substring in ["_c01_","_c02_","_c03_","_c04_","_c05_","_c06_","_c07_","_c08_", "_c09_"]:
    string = string.replace(substring,"_cAll_")
  return folder + string + ".pkl"

def image2tensor(file_path):
  """
  Given an image file path returns a tensor of the image
  """
  # Reads image
  image = imread(file_path)
  # Downsamples by 4 to go from 1080x1920 to 270x480
  resized = image[::4, ::4, :]
  # Converts numpy array to tensor
  tensor = tf.convert_to_tensor(resized, dtype=tf.float32)
  # Shape is (270, 480, 3)
  return tensor

def check_num_frames(frame_tensor, keypoint_tensor):
  if keypoint_tensor.shape[0] < frame_tensor.shape[0]:
    num_to_cut = frame_tensor.shape[0] - keypoint_tensor.shape[0] 
    frame_tensor = frame_tensor[0:-num_to_cut,:,:,:]
  elif keypoint_tensor.shape[0] > frame_tensor.shape[0]:
    num_to_cut = - frame_tensor.shape[0] + keypoint_tensor.shape[0]
    keypoint_tensor = frame_tensor[0:-num_to_cut,:,:]
  return frame_tensor, keypoint_tensor

# Read file contents and put each line into a list
def read_files_from_list(text_destination):
  dis_list = []
  with open(text_destination, 'r') as f:
    for line in f:
      nonewline = line[:-1]       # Removes the \n from the end of each line
      dis_list.append(nonewline)  # Add file to list
  return dis_list

# Write and create a text file that prints some statement (will overwrite if file exists)
def write_list_to_file(dis_list, file_path):
  with open(file_path, 'w') as f:
    for item in dis_list:
      print(item, file=f)


def update_vids_converted_file(new_vids, file_path = "/content/drive/My Drive/CS230Project/data/vids_converted_to_data.txt"):
  f = open(file_path, "a")
  f.close()
  vid_list_old = read_files_from_list(file_path)
  vid_list = vid_list_old + new_vids
  write_list_to_file(vid_list, file_path)


def get_list_of_frames(images_folder, labels_folder, visited_list, num_videos = 200,
  visited_list_file_destination = "/content/drive/My Drive/CS230Project/framesInfo/converted_tensor_list.txt"):
  first_tensor = True
  first_vid = True
  j = 0
  frame_tensor_list = []
  keypoints_list = []
  root_list = []

  for root, dirs, files in os.walk(images_folder, topdown=True): 
    if root in visited_list:
      continue
    elif j == 0:
      j += 1
      # Skip the first iteration of os.walk since it starts with the root folder
      continue
    elif j % 10 == 0:
      print(f"finished {j} iterations")
    elif j > num_videos:
      break
    j += 1

    # Add to visited list
    visited_list.append(root)
    root_list.append(root)

    #pickle stuff
    pickle_file = vidpath2picklepath(labels_folder, root)
    keypoints_list.append(pklpath2tensor(pickle_file))

    for file in files:
      # Get the tensor from the file given in the root path
      frame_tensor = image2tensor(root + "/" + file)

      if first_tensor:
        first_tensor = False   # Toggle boolean so this block only runs once

        # Makes the first tensor of dimension (1, length, width, channel)
        all_tensor_frames = tf.expand_dims(frame_tensor, axis=0)

      else:
        # Makes the next tensor frame of dimension (1, length, width, channel)
        next_tensor_frame = tf.expand_dims(frame_tensor, axis=0)
        # Concatenates onto the rest of the tensors
        all_tensor_frames = tf.concat([all_tensor_frames, next_tensor_frame], axis=0)

    #add tensor to tensor list
    frame_tensor_list.append(all_tensor_frames / 255.0)
    first_tensor = True

  # match the frame number for each video
  print("checking")
  for i in range(len(frame_tensor_list)):
    frame_tensor_list[i] ,keypoints_list[i] = check_num_frames(frame_tensor_list[i], keypoints_list[i])

  # Write the visited list to a file for long term storage
  write_list_to_file(visited_list, visited_list_file_destination)
  update_vids_converted_file(root_list)
  return frame_tensor_list , keypoints_list, root_list

def savedata2file(frame_tensor_list, keypoints_list, root_list, file_path = '/content/drive/My Drive/CS230Project/data/MEGADATAFILE.hdf5'):
  with h5py.File(file_path, 'a') as hf:
    for i in range(len(root_list)):
      grp = hf.create_group(root_list[i])
      grp.create_dataset('frames',  data = frame_tensor_list[i])
      grp.create_dataset('keypoints', data = keypoints_list[i])


#### old get samples
def get_samples_h5py(h5py_path, num_videos, finished_samples, vid_name_list_path = "/content/drive/My Drive/CS230Project/data/vids_converted_to_data.txt"):
  vid_name_list = read_files_from_list(vid_name_list_path)
  vid_name_list = vid_name_list[finished_samples:]
  first_iter = True
  with h5py.File(h5py_path, 'r') as hf:
    for i, video in enumerate(vid_name_list):
      if i >= num_videos:
        break
      if first_iter:
        image_set = hf[video + '/frames'][:]
        keypoint_set = hf[video + '/keypoints'][:]
        first_iter = False
      else:
        image_set = tf.concat([image_set, hf[video + '/frames'][:]], axis = 0)
        keypoint_set = tf.concat([keypoint_set, hf[video + '/keypoints'][:]], axis = 0)
      finished_samples += 1
  keypoint_set = tf.reshape(keypoint_set, [keypoint_set.shape[0], -1])
  return image_set, keypoint_set, finished_samples



def get_samples_new(frames_path, labels_folder, batch_size, only_one_sample=True, video_index=0, frame_index=0):
  # frames_path = file path to downsampled folder of the video frames
  # labels_folder = file path to keypoints folder of the all the pickle lables
  # image_dims = (height, width, channel) of an image
  # batch_size = number of training examples per training pass
  # bundle_size = Number of frames we take to group together to run as a time series

  image_height = 270
  image_width = 480
  image_channel = 3

  # image_height, image_width, image_channel = image_dims
  video_names = os.listdir(frames_path) # frames path contains folders named with each video containing the frames for that video
  result_array = np.zeros(shape=(batch_size, image_height, image_width, image_channel))
  labels_array = np.zeros(shape=(batch_size, 51))
  # labels_array_notvec = np.zeros(shape=(batch_size, 17,3))

  # If we only care about getting a single random sample, then just pick from the list randomly
  if only_one_sample:
    video = np.random.choice(video_names)
  # If we want to pick repeatedly without replacement, then pick up from given video index and frame index
  else:
    video = video_names[video_index]
    
  frames_per_video = frames_path + video # path to the directory containing the frames of a given video
  frames = os.listdir(frames_per_video) # Gets the list of videonameframe000x.jpg'
  num_frames = len(frames)

  for i in range(batch_size):
    # If the frame index exceeds the number of frames left
    # while frame_index >= num_frames:  #original line of code
    while frame_index >= num_frames - 2: #hacky line of code to account for mismatched labels and frames
      # Reset variables for next video
      video_index += 1
      # Handles the case of running out of videos (returns None)
      try:
        video = video_names[video_index]
      except:
        return None
      frames_per_video = frames_path + video # path to the directory containing the frames of a given video
      frames = os.listdir(frames_per_video) # Gets the list of videonameframe000x.jpg's
      frame_index = 0     # Clear frame index
      num_frames = len(frames)
    # print('frame index:', frame_index, "num_frames:", num_frames)

    # Create an empty tensor for the each training example
    # frame = frames[frame_index] #original line of code
    frame = frames[frame_index ] #hacky line of code to account for mismatched labels and frames
    image = imread(frames_per_video + "/" + frame)

    result_array[i, :, :, :] = image/255  # Set inner array to outer array

    #pickle stuff
    pickle_file = vidpath2picklepath(labels_folder, frames_per_video) 
    keypoints_tensor = pklpath2tensor(pickle_file)
    # print(keypoints_tensor.shape)
    keypoint_set = keypoints_tensor[frame_index - 1, :, :]
    label = tf.reshape(keypoint_set, [-1])
    labels_array[i, :] = label
    # labels_array_notvec[i,:,:] = keypoint_set
    
    frame_index += 1  # Increment the frame index for the next bundle of frames

    # Convert to tensor from np array
    result_tensor = tf.convert_to_tensor(result_array, dtype=tf.float32)
    labels_tensor = tf.convert_to_tensor(labels_array, dtype=tf.float32)

  # return result_tensor, labels_array, video_index, frame_index
  return result_tensor, labels_tensor, video_index, frame_index


In [3]:
######### Model ###############


def create_model(lossfunc = 'mae', optimizer = 'adam', metric = 'mae', num_frames_per_block = 10, output_mode = 'allframes'):

  initializer = tf.keras.initializers.GlorotNormal()

  model = models.Sequential()
  model.add(layers.Conv3D(
    filters = 32, kernel_size = (1,1,1), input_shape=(480, 270, num_frames_per_block, 3), strides=(1, 1, 1), name = 'layer1',padding='valid',
    data_format=None, dilation_rate=(1, 1, 1), groups=1, activation='relu',
    use_bias=True, kernel_initializer=initializer,
    bias_initializer='zeros', kernel_regularizer=None,
    bias_regularizer=None, activity_regularizer=None, kernel_constraint=None,
    bias_constraint=None
  ))
  model.add(layers.MaxPooling3D((2, 2, 1), name = 'layer2'))
  model.add(layers.Conv3D(32, (3, 3, 3), activation='relu', kernel_initializer=initializer, name = 'layer3'))
  model.add(layers.MaxPooling3D((2, 2,1), name = 'layer4'))
  model.add(layers.Conv3D(64, (3, 3,3), activation='relu', kernel_initializer=initializer, name = 'layer5'))
  model.add(layers.MaxPooling3D((2, 2,1), name = 'layer6'))
  model.add(layers.Conv3D(64, (5, 5,5), activation='relu', kernel_initializer=initializer, name = 'layer7'))
  model.add(layers.MaxPooling3D((3, 3,1), name = 'layer8'))
  model.add(layers.Conv3D(16, (1, 1,1), activation='relu', kernel_initializer=initializer, name = 'layer9'))
  model.add(layers.Flatten(name = 'layer10')) #flatten into 1D vector
  if output_mode == 'allframes':
    model.add(layers.Dense(51*num_frames_per_block, kernel_initializer=initializer, name = 'layer11'))
  elif output_mode == 'midframe':
    model.add(layers.Dense(51, kernel_initializer=initializer, name = 'layer11'))

  model.compile(optimizer=optimizer,loss= lossfunc,metrics=[metric])
  

  return model

In [ ]:
#@title Update Model function pulling from  h5py 
def update_model_h5(num_vids_in_train_set = 5, num_vids_in_test_set = 2, epochs = 5, lossfunc = 'mae', optimizer = 'adam', metric = 'mae', weights_path = 'latest_update'):
  print('\n\n')
  print("Model Updating")
  checkpoint_path = f"/content/drive/MyDrive/CS230Project/regular_model_checkpoints/cp_{datetime.now()}.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  # Create a callback that saves the model's weights every 5 epochs
  cp_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_path, 
      verbose=1, 
      save_weights_only=True,
      save_freq='epoch',
      save_best_only=True)


  #### Fit model
  model = create_model(lossfunc, optimizer , metric)

  if weights_path == 'latest_update':
    latest_update = tf.train.latest_checkpoint(checkpoint_dir)
    model.load_weights(latest_update)
  else:
    model.load_weights(weights_path)


  #get data
  h5py_path = '/content/drive/My Drive/CS230Project/data/MEGADATAFILE.hdf5'
  labels_folder = '/content/drive/My Drive/CS230Project/pkl_files/aist_plusplus_final/keypoints3d/'

  #get from file
  with open('/content/drive/MyDrive/CS230Project/regular_model_checkpoints/indices/h5_index_file.csv') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      for row in csv_reader:
          finished_samples = int(row[0])
  print('start_samplenum:', finished_samples)

  train_images, train_labels, finished_samples = get_samples_h5py(h5py_path, num_vids_in_train_set, finished_samples)
  test_images, test_labels, finished_samples = get_samples_h5py(h5py_path, num_vids_in_test_set, finished_samples)

  # #### manually reset start indices if you so desire. dont do this unless you have a good reason
  with open('/content/drive/MyDrive/CS230Project/regular_model_checkpoints/indices/h5_index_file.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow([finished_samples])  

  print('end_samplenum:', finished_samples)

  # Train the model with the new callback
  history = model.fit(train_images, 
            train_labels,
            epochs=epochs, 
            callbacks=[cp_callback],
            validation_data=(test_images, test_labels),
            verbose=0)
  return model, history, train_images, train_labels, test_images, test_labels

In [9]:
#@title Global variabl for persistent test set from video 7500 index 0 400 samples
downsampled_frames_folder = "/content/drive/My Drive/CS230Project/downsampled/"
labels_folder = '/content/drive/My Drive/CS230Project/pkl_files/aist_plusplus_final/keypoints3d/'
test_images_persistent, test_labels_persistent, _, _ = get_samples_new(downsampled_frames_folder, labels_folder, 15, False, 3065, 16)
# test_images_persistent = tf.expand_dims(test_images_persistent, axis = 3)

In [ ]:
# @title recover best path and loss
# # '/content/drive/My Drive/CS230Project/checkpoints/regular_1000ex_mae6'
# model = create_model(lossfunc = 'mae', optimizer = 'adam' , metric = 'mae')
# default_path = '/content/drive/My Drive/CS230Project/checkpoints/regular_1000ex_mae6'
# checkpoint_path = f"/content/drive/MyDrive/CS230Project/regular_model_checkpoints/cp_auto_{datetime.now()}.ckpt"
# checkpoint_dir = os.path.dirname(checkpoint_path)
# auto_path = tf.train.latest_checkpoint(checkpoint_dir)

# with open('/content/drive/MyDrive/CS230Project/best_stuff_3d/best_stuff.csv') as csv_file:
#     csv_reader = csv.reader(csv_file, delimiter=',')
#     for row in csv_reader:
#       best_path, best_loss = row[0], float(row[1])

# path_testing = auto_path

# model.load_weights(path_testing)
# loss, acc = model.evaluate(test_images_persistent, test_labels_persistent, verbose=2)
# with open('/content/drive/MyDrive/CS230Project/best_stuff_3d/best_stuff.csv', mode='w') as csv_file:
#   writer = csv.writer(csv_file, delimiter=',')
#   writer.writerow([path_testing, loss  ]) 

In [10]:
#@title Update model function pulling from image folders ####################################################
def reshapeinput_conv2d_to_conv3d(input_tensor, labels_tensor, num_frames_per_block = 10, output_mode = 'allframes'):
    i = 0
    outputlist = []
    labellist = []
    while input_tensor.shape[0] >= (i+1)*num_frames_per_block:
      outputlist.append(tf.transpose(input_tensor[i*num_frames_per_block:(i+1)*num_frames_per_block,:,:,:], perm =[2,1,0,3] ))
      if output_mode == 'allframes':
        labellist.append(tf.reshape(labels_tensor[i*num_frames_per_block:(i+1)*num_frames_per_block,:],[-1]))
      elif output_mode == 'midframe':
        labellist.append(labels_tensor[int((i*num_frames_per_block + (i+1)*num_frames_per_block)/2),:])
      i += 1
    return tf.stack(outputlist), tf.stack(labellist)


def update_model(num_train_frames = 200, num_test_frames = 5, num_frames_per_block = 10, output_mode = 'allframes', epochs = 5, lossfunc = 'mae', optimizer = 'adam', metric = 'mae', weights_path = 'best_manual_update'):
  print('\n\n')
  print("Model Updating")
  checkpoint_path = f"/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/cp_auto_{datetime.now()}.ckpt"
  checkpoint_dir = os.path.dirname(checkpoint_path)

  #Create a callback that saves the model's weights every 5 epochs
  cp_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_path, 
      verbose=1, 
      save_weights_only=True,
      save_freq='epoch',
      save_best_only=True)


  #### Fit model
  model = create_model(lossfunc, optimizer , metric, num_frames_per_block, output_mode = output_mode)


  with open('/content/drive/MyDrive/CS230Project/best_stuff_3d/best_stuff.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
      best_path, best_loss = row[0], float(row[1])

  if weights_path == 'best_auto_update':
    load_path = tf.train.latest_checkpoint(checkpoint_dir)
  elif weights_path == 'best_manual_update':
    load_path = best_path
    load_path = "/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/cp_auto_2021-06-02 05:26:31.936953.ckpt"
    # load_path = "/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/cp_manual_2021-03-16 18:40:32.893846"
    # /content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/cp_manual_2021-06-01 06:37:19.052223
  else:
    load_path = weights_path
  
  if weights_path != 'new_initialization':
    model.load_weights(load_path)
    print("model loaded from", load_path)
  else:
    print('new model weights initialized')



  #get data
  downsampled_frames_folder = "/content/drive/My Drive/CS230Project/downsampled/"
  labels_folder = '/content/drive/My Drive/CS230Project/pkl_files/aist_plusplus_final/keypoints3d/'

  #get from file
  with open('/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/indices/current_index_file.csv') as csv_file:
      csv_reader = csv.reader(csv_file, delimiter=',')
      for row in csv_reader:
          vid_idx, frame_idx = int(row[0]), int(row[1])
  
  print('start_indices:', vid_idx, frame_idx)
  #test and train on same data because theres a separate test set outside of this function
  train_images, train_labels, vid_idx, frame_idx = get_samples_new(downsampled_frames_folder, labels_folder, num_train_frames, False, vid_idx, frame_idx)
  test_images, test_labels, vid_idx, frame_idx = train_images, train_labels, vid_idx, frame_idx#get_samples_new(downsampled_frames_folder, labels_folder, num_test_frames, False,  vid_idx, frame_idx)

  with open('/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/indices/current_index_file.csv', mode='w') as csv_file:
      writer = csv.writer(csv_file, delimiter=',')
      writer.writerow([vid_idx, frame_idx])

  print('end_indices:', vid_idx, frame_idx)

  test_images_reshaped, test_labels_reshaped = reshapeinput_conv2d_to_conv3d(test_images,test_labels,num_frames_per_block, output_mode = output_mode)
  train_images_reshaped, train_labels_reshaped = reshapeinput_conv2d_to_conv3d(train_images, train_labels, num_frames_per_block, output_mode = output_mode)

  print('input shapes')
  print(train_images_reshaped.shape)
  print(train_labels.shape)
  print(train_labels_reshaped.shape)
  # Train the model with the new callback
  history = model.fit(train_images_reshaped, 
            train_labels_reshaped,
            epochs=epochs, 
            # callbacks=[cp_callback],
            validation_data=(test_images_reshaped, test_labels_reshaped),
            verbose=True)

  test_images_persistent_reshaped, test_labels_persistent_reshaped = reshapeinput_conv2d_to_conv3d(test_images_persistent,test_labels_persistent,num_frames_per_block, output_mode = output_mode)
  loss, acc = model.evaluate(test_images_persistent_reshaped, test_labels_persistent_reshaped, verbose=2)


  
  if loss < best_loss:
    print(f"loss on test set of {loss} improves on previous loss of {best_loss}")
    best_loss = loss
    best_path = f"/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/cp_manual_{datetime.now()}"
    model.save_weights(best_path)
    print(f"new best model stored in", best_path)

    with open('/content/drive/MyDrive/CS230Project/best_stuff_3d/best_stuff.csv', mode='w') as csv_file:
      writer = csv.writer(csv_file, delimiter=',')
      writer.writerow([best_path, best_loss])  
  else:
    print(f"loss on test set of {loss} does not improve on previous loss of {best_loss}. Model not manually saved") 

  return model, history, train_images, train_labels, test_images, test_labels


In [14]:
#@title reset best path and loss to reset training
best_path = 'null'
best_loss = 100
with open('/content/drive/MyDrive/CS230Project/best_stuff_3d/best_stuff.csv', mode='w') as csv_file:
      writer = csv.writer(csv_file, delimiter=',')
      writer.writerow([best_path, best_loss])

In [11]:
# hyperparameters
# weights_path = 'new_initialization'
weights_path = 'best_auto_update'
lossfunc = 'mae'
metric = 'mae'
output_mode = 'allframes'
optimizer = 'adam'
# output_mode = 'midframe'

num_train_frames = 15 #must be multiple of num_frames_per_block
num_test_frames =15 
num_frames_per_block = 15
epochs = 30
loop_iter = 1

In [ ]:
##@title Update model pulling images from files ###################################################
for k in range(loop_iter): 

  manual_reset_vid_index = 3065 + k
  manual_reset_frame_index = 0
  with open('/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/indices/current_index_file.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow([manual_reset_vid_index, manual_reset_frame_index])

  ################## run model ######################
  print('num_frames_per_block:', num_frames_per_block)
  print('num_train_frames:', num_train_frames)
  print('output_mode:', output_mode)

  start = time.time() 
  model, history, train_images, train_labels, test_images, test_labels = update_model( num_train_frames = num_train_frames,  num_test_frames = num_test_frames, num_frames_per_block = num_frames_per_block, output_mode = output_mode, epochs = epochs, lossfunc = lossfunc, metric = metric, weights_path = weights_path)
  end = time.time() 
  print(f"Runtime of the program is {end - start}")
  ################# plot model history ###############
  plt.figure()
  plt.plot(history.history[metric], label=metric)
  plt.xlabel('Epoch')
  plt.ylabel(metric)
  plt.legend(loc='lower right') 


  ################### plot 2d projection of individual samples vs test set #################

  # train_images_reshaped = reshapeinput_conv2d_to_conv3d(train_images,num_frames_per_block)
  # predicted_y = model.predict(train_images_reshaped)
  # output_pts = predicted_y.reshape((predicted_y.shape[0],17,int(predicted_y.shape[1]/17)))
  # label_pts = train_labels.numpy().reshape((train_labels.shape[0],17,int(predicted_y.shape[1]/17)))

  # for i in range(10):
  #     # i = int(test_images.shape[0]/2)
  #     plt.figure()
  #     plt.scatter(output_pts[i,:,0],output_pts[i,:,1], color = 'red')
  #     plt.scatter(label_pts[i,:,0],label_pts[i,:,1], color = 'blue')
  #     plt.title('Sample output plot. Model predictions in blue, Ground truth in red')
  #     plt.show()

  # test_images_reshaped, test_labels_reshaped = reshapeinput_conv2d_to_conv3d(test_images,test_labels, num_frames_per_block , output_mode = output_mode)
  # predicted_y = model.predict(test_images_reshaped)
  # if output_mode == 'allframes':
  #   output_pts = predicted_y.reshape((predicted_y.shape[0],17,num_frames_per_block,-1))
  #   label_pts = test_labels_reshaped.numpy().reshape((test_labels_reshaped.shape[0],17,num_frames_per_block,-1))
  # elif output_mode == 'midframe':
  #   output_pts = predicted_y.reshape((predicted_y.shape[0],17,1,-1))
  #   label_pts = test_labels_reshaped.numpy().reshape((test_labels_reshaped.shape[0],17,1,-1))

  # for i in range(int(num_train_frames/num_frames_per_block)):
  #     # i = int(test_images.shape[0]/2)
  #     # for j in range(num_frames_per_block):
  #     for j in range(1):
  #       print(f'train set loop {k} fig {j}')
  #       plt.figure()
  #       plt.scatter(output_pts[i,:,j,0],output_pts[i,:,j,1], color = 'red')
  #       plt.scatter(label_pts[i,:,j,0],label_pts[i,:,j,1], color = 'blue')
  #       plt.title('Conv3d sample output keypoints. Model predictions in blue, Ground truth in red')
  #       plt.show()

  # ################### plot 2d projection of individual samples vs test set #################
  # predicted_y = model.predict(test_images)
  # output_pts = predicted_y.reshape((predicted_y.shape[0],17,int(predicted_y.shape[1]/17)))
  # label_pts = test_labels.numpy().reshape((test_labels.shape[0],17,int(predicted_y.shape[1]/17)))

  # for i in range(10):
  #     i = int(test_images.shape[0]/2)
  #     plt.figure()
  #     plt.scatter(output_pts[i,:,0],output_pts[i,:,1], color = 'red')
  #     plt.scatter(label_pts[i,:,0],label_pts[i,:,1], color = 'blue')
  #     plt.title('Sample output plot. Model predictions in blue, Ground truth in red')
  #     plt.show()


In [ ]:

################### plot 2d projection of individual samples vs persistent test set #################
model = create_model(lossfunc, optimizer , metric, num_frames_per_block, output_mode = output_mode)
checkpoint_path = f"/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/cp_auto_{datetime.now()}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
load_path = tf.train.latest_checkpoint(checkpoint_dir)

with open('/content/drive/MyDrive/CS230Project/best_stuff_3d/best_stuff.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
      best_path, best_loss = row[0], float(row[1])


load_path = best_path
print('loaded from:',load_path)
model.load_weights(load_path)


test_images_reshaped, test_labels_reshaped = reshapeinput_conv2d_to_conv3d(test_images_persistent,test_labels_persistent, num_frames_per_block , output_mode = output_mode)

start = time.time()
predicted_y = model.predict(test_images_reshaped)
end = time.time()
print(f"Runtime of the program is {end - start}")


if output_mode == 'allframes':
  output_pts = predicted_y.reshape((predicted_y.shape[0],17,num_frames_per_block,-1))
  label_pts = test_labels_reshaped.numpy().reshape((test_labels_reshaped.shape[0],17,num_frames_per_block,-1))
elif output_mode == 'midframe':
  output_pts = predicted_y.reshape((predicted_y.shape[0],17,1,-1))
  label_pts = test_labels_reshaped.numpy().reshape((test_labels_reshaped.shape[0],17,1,-1))

label_pts = test_labels_persistent.numpy().reshape((test_labels_persistent.shape[0],17,-1))

for i in range(int(num_train_frames/num_frames_per_block)):
    # i = int(test_images.shape[0]/2)
    # for j in range(num_frames_per_block):





    for j in range(num_frames_per_block):

      plt.figure()
      plt.imshow(test_images_persistent.numpy()[j,:,:,:], interpolation='nearest')
      plt.title(f"Test image {j}")
      plt.show()


      print(f'test set loop  fig {j}')
      plt.figure()
      plt.scatter(output_pts[i,:,j,0],output_pts[i,:,j,1], color = 'red')
      # plt.scatter(label_pts[i,:,j,0],label_pts[i,:,j,1], color = 'blue')
      plt.scatter(label_pts[j,:,0],label_pts[j,:,1], color = 'blue')
      plt.title('Conv3d sample output keypoints. Model predictions in blue, Ground truth in red')
      plt.show()

In [ ]:
#@title h5py hyperparameters and default weight reload (these are together for no reason)
lossfunc = 'mae'
metric = 'mae'
num_vids_in_train_set = 5
num_vids_in_test_set = 1
epochs = 5
loop_iter = 10

# '/content/drive/My Drive/CS230Project/checkpoints/regular_1000ex_mae6'
model = create_model(lossfunc = 'mae', optimizer = 'adam' , metric = 'mae')
default_path = '/content/drive/My Drive/CS230Project/checkpoints/regular_1000ex_mae6'
model.load_weights(default_path)
loss, acc = model.evaluate(test_images_persistent, test_labels_persistent, verbose=2)
with open('/content/drive/MyDrive/CS230Project/best_stuff/best_stuff.csv', mode='w') as csv_file:
  writer = csv.writer(csv_file, delimiter=',')
  writer.writerow([default_path, loss  ]) 

In [ ]:
#@title Update model pulling images from h5py ###################################################
h5 = True
for i in range(loop_iter): 
  ################## run model ######################
  if h5:
    model, history, train_images, train_labels, test_images, test_labels = update_model_h5(num_vids_in_train_set = num_vids_in_train_set, num_vids_in_test_set = num_vids_in_test_set, epochs = epochs, lossfunc = lossfunc, metric = metric)#, weights_path = '/content/drive/MyDrive/CS230Project/regular_model_checkpoints/cp_2021-03-16 03:02:02.611619.ckpt')
  else:
    model, history, train_images, train_labels, test_images, test_labels = update_model( num_train_frames = num_train_frames,  num_test_frames = num_test_frames, epochs = epochs, lossfunc = lossfunc, metric = metric)
  
  ################# plot model history ###############
  plt.plot(history.history[metric], label=metric)
  plt.xlabel('Epoch')
  plt.ylabel(metric)
  plt.legend(loc='lower right') 


  ################### plot 2d projection of individual samples vs test set #################
  predicted_y = model.predict(test_images)
  output_pts = predicted_y.reshape((predicted_y.shape[0],17,int(predicted_y.shape[1]/17)))
  label_pts = test_labels.numpy().reshape((test_labels.shape[0],17,int(predicted_y.shape[1]/17)))

  for i in range(1):
      plt.figure()
      plt.scatter(output_pts[i,:,0],output_pts[i,:,1], color = 'red')
      plt.scatter(label_pts[i,:,0],label_pts[i,:,1], color = 'blue')
      plt.title('Sample output plot. Model predictions in blue, Ground truth in red')
      plt.show()

In [ ]:
#@title plot model history ###############
plt.plot(history.history[metric], label=metric)
plt.xlabel('Epoch')
plt.ylabel(metric)
plt.legend(loc='lower right')

In [ ]:
#@title plot individual samples vs test set #################
predicted_y = model.predict(train_images)
output_pts = predicted_y.reshape((predicted_y.shape[0],17,3))
label_pts = test_labels.numpy().reshape((test_labels.shape[0],17,3))

#2d plots of outputs and labels of selected images 
for i in range(1):
    i = 2
    plt.figure()
    plt.scatter(output_pts[i,:,0],output_pts[i,:,1], color = 'red')
    plt.scatter(label_pts[i,:,0],label_pts[i,:,1], color = 'blue')
    plt.title('Sample output plot. Model predictions in blue, Ground truth in red')
    plt.show()


In [ ]:
#@title  manually reset start indices if you so desire. dont do this unless you have a good reason
manual_reset_vid_index = 20
manual_reset_frame_index = 0
with open('/content/drive/MyDrive/CS230Project/regular_3d_model_checkpoints/indices/current_index_file.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow([manual_reset_vid_index, manual_reset_frame_index])

In [ ]:
#@title h5py manually reset vid_number if you so desire. dont do this unless you have a good reason
manual_num_vids = 48
with open('/content/drive/MyDrive/CS230Project/regular_model_checkpoints/indices/h5_index_file.csv', mode='w') as csv_file:
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow([manual_num_vids])